In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import json
import seaborn as sns
sys.path.append('/home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/')
#from src.utils.logger_functions import console
from src.cmos_plotter.Plotter_Helper_KV import *
from src.utils.metadata_functions import load_metadata_as_dataframe
from src.utils.logger_functions import console
from src.cmos_plotter import Conduction_speed_plotter as csp
import spikeinterface.extractors as se
import pickle
import matplotlib.pyplot as plt
from matplotlib import cm
import logging
# Silence just the matplotlib.font_manager logger
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

In [ ]:
MAIN_PATH = '/itet-stor/kvulic/neuronies/gbm_project/1_Subprojects/Information_Capacity/Recordings/Patient_1'

RECORDINGS_PATH = os.path.join(MAIN_PATH,f'Raw_Traces')
SORTER_PATH = os.path.join(MAIN_PATH, f'Processed_Data/ID1814_Patient1')
CONDUCTION_SPEEDS_PATH = '/itet-stor/kvulic/neuronies/kvulic_project1/00_temp_GA'

if not os.path.exists(CONDUCTION_SPEEDS_PATH):
    os.makedirs(CONDUCTION_SPEEDS_PATH)

metadata = load_metadata_as_dataframe(file_path=RECORDINGS_PATH)

In [ ]:
results_combined = []
for idx, filename in enumerate(metadata.Filename):
    chip_id = metadata.loc[idx, 'Chip_ID']  # Get specific row values
    div = metadata.loc[idx, 'DIV']
    #cell_type = metadata.loc[idx, 'Cell_Type']
    cell_type = 'NGN2'
    try:
        print('Calculating conduction speeds for:', filename)
        recording = se.MaxwellRecordingExtractor(os.path.join(RECORDINGS_PATH, f'{filename}'))
        template_path = os.path.join(SORTER_PATH, f'Sorter_{filename}/wf_folder_curated/')
        
        sampling_rate = recording.get_sampling_frequency()
        probe_locations = recording.get_channel_locations()
        templates = csp.load_and_extract_templates(template_path)
        results = csp.analyze_conduction_speeds(templates, probe_locations, sampling_rate)
        
        # Add metadata and rename speed column for each result
        for result in results:
            result['speed_ms-1'] = result.pop('speed') / 1000  # Convert to m/s and rename
            result['chip_id'] = chip_id
            result['div'] = div
            result['cell_type'] = cell_type
            result['filename'] = filename
        
        
        if len(results) != 0:
            results_combined = results_combined + results


    except Exception as e:
        console.info(f'Could not calculate conduction speeds for {filename} due to the following error: {e}')
        continue


In [ ]:
# Save the combined results at the end
with open(os.path.join(CONDUCTION_SPEEDS_PATH, f'All_conduction_speeds.pkl'), 'wb') as f:
    pickle.dump(results_combined, f)

#save results to csv
results_combined_df = pd.DataFrame(results_combined)
results_combined_df.to_csv(os.path.join(CONDUCTION_SPEEDS_PATH, f'All_conduction_speeds.csv'), index=False)
print("Done!")

In [ ]:
csp.visualize_unit_propagation_V2(CONDUCTION_SPEEDS_PATH, filename, results, templates, probe_locations, 
                              sampling_rate, figsize=(10, 8), fontsize=12, unit_ids=None)